In [5]:
import advent
data = advent.get_char_grid(21, 'txt')

In [8]:
from advent.maze import solve_maze_no_tqdm
import numpy as np

def walkable(node: tuple[int, int]) -> bool:
    return node[0] >= 0 and node[0] < data.shape[0] and node[1] >= 0 and node[1] < data.shape[1] and data[node] != '#'

def adjacent(node: tuple[int, int]) -> list[tuple[tuple[int, int], int]]:
    x, y = node
    return [((x+a, y+b), 1) for a in range(-1, 2) for b in range(-1, 2) if (a != 0) != (b != 0) and walkable((x+a, y+b))]

start = np.where(data == 'S')[0][0], np.where(data == 'S')[1][0]

In [12]:
_, parents, _ = solve_maze_no_tqdm(start, (lambda _: False), adjacent)

def get_shortest_path_length(node: tuple[int, int]) -> int:
    result = 0
    while node != start:
        node = parents[node]
        result += 1
    return result

# Reason this works:
# Any node you can reach in LESS than 64 steps (e.g. 60) you can also reach in 64
# By going to the node, and then just stepping off-and-on it every 2 steps
# However, a node you can reach in an odd number of steps can NEVER be reached in 64 steps
# So the formula becomes: shortest path <= 64 and shortest path % 2 == 0
def get_reachable(max: int = 64, rem: int = 0):
    result = 0
    for node in parents:
        l = get_shortest_path_length(node)
        if l <= max and l % 2 == rem: result += 1
    return result

get_reachable()

3782

In [21]:
# Part 2 approach:
# I didn't like this one... I was overthinking it too much. With some pen and paper I ended up coming up with a simpler solution:
# let □(ws) be all spots reachable in odd steps from S
# let ■(bs) be all spots reachable in even steps from S
# let ◇(wd) be all spots reachable in odd steps from S, EXCLUDING the center diamond
# let ◆(bd) be all spots reachable in even steps from S, EXCLUDING the center diamond
# let N be (26501365-65) / 131, where 131 is the grid size, and 65 is half the grid size
# then the solution is:

def solution(ws: int, bs: int, wd: int, bd: int, N: int):
    return ((N+1)**2 * ws) + (N**2 * bs) + (N * bd) - ((N+1) * wd)

# ws and bs is just anything that can be reached in any number of steps, we just need to do the parity check
ws = get_reachable(1000, 1)
bs = get_reachable(1000, 0)

# wd and bd can be calculated by taking ws or bs, and subtracting anything reachable in 65 steps (aka the center)
wd = ws - get_reachable(65, 1)
bd = bs - get_reachable(65, 0)

N = (26501365 - data.shape[0] // 2) // data.shape[0]

solution(ws, bs, wd, bd, N)

630661863455116

In [22]:
# small explanation how I came up with it:
# I quickly figured out the verticals/horizontals were empty, meaning the shortest path would be going straight for most the route,
# meaning I only had to 'maze solve' the edges. I overthought this part, thinking I had to do maze solving on every quadrant from every possible corner
# Which would be like a dozen maze solves to keep track of, and a lot of bookkeeping to count which 'type' of edge quadrant you are approaching
# I then got a hint, which is that somebody solved it in like 10-20 lines of code, meaning there must have been a simpler way.
# I then got a piece of pen and paper and drew it out, and then it 'clicked' that you could just count each quadrant separately
# The formula in the `solution` function was derived by drawing out the entire scenario for N=3, which made it fairly obvious what the generalized formula should be